In [ ]:
import iris
import param
import numpy as np
import holoviews as hv
import holocube as hc
from matplotlib.cm import cmap_d
from cartopy import crs as ccrs
from cartopy import feature as cf
from paramnb import NbParams, FileSelector

hv.notebook_extension(width=90)
%output widgets='live' size=400
%opts Image {+framewise} [colorbar=True] Contours {+framewise}

Declaring a set of widgets using paramNB is as simply as declaring a Parameterized class. Here we declare a ``CubeLoader`` class with a ``FileSelector`` parameter, a cache and a load method, which will load cubes from a file on disk. The ``NbParams`` function accepts the ``CubeLoader`` instance and generates a dropdown widget for the ``cube_path`` from it. By declaring a callback we can tell it to load the file from disk (or the cache) whenever the user executes the widget. Finally we can declare the ``execute`` mode, which allows declaring no execution with ``execute=None`` or adds a button to execute the 'next' cell or all cells 'below'.

In [ ]:
class CubeLoader(param.Parameterized):
    
    cube_path = FileSelector(default='files/*.pp')
    
    def __init__(self, **params):
        super(CubeLoader, self).__init__(**params)
        self.cache = {}
        self.cubes = None
    
    def load(self, cube_loader):
        if self.cube_path not in self.cache:
            cubelist = iris.load(self.cube_path)
            for c in cubelist:
                c.coord('grid_longitude').guess_bounds()
                c.coord('grid_latitude').guess_bounds()
            self.cache[self.cube_path] = cubelist
        else:
            cubelist = self.cache[self.cube_path]
        cubes = {cb.vdims[0].name:cb for cb in [hc.HoloCube(c) for c in cubelist]} # Load cubes into dictionary
        self.cubes = {k:v for k,v in cubes.items() if k!='unknown'}  # Filter as desired

cube_loader = CubeLoader()
NbParams(cube_loader, execute='next', callback=cube_loader.load)

In [ ]:
# Next we declare a more complex ``CubeBrowser`` class, which will define the various options on what to display and how to display it.
# We add parameters to control the cmap, quantity, element and projection and populate these with useful options.

def all_subclasses(cls):
    return cls.__subclasses__() + [g for s in cls.__subclasses__()
                                   for g in all_subclasses(s)]

projections = {crs.__name__: crs for crs in all_subclasses(ccrs.CRS)
               if hasattr(crs, '_as_mpl_axes') and not crs.__name__[0] == '_'}

class CubeBrowser(param.Parameterized):

    cmap = param.ObjectSelector(default='viridis', objects=list(cmap_d.keys()))

    quantity = param.ObjectSelector(default=cube_loader.cubes.keys()[0], objects=list(cube_loader.cubes.keys()))

    element = param.ObjectSelector(default=hc.Image, objects=[hc.Image, hc.Contours])

    projection = param.ObjectSelector(default='PlateCarree', objects=list(projections.keys()))
    
    cache = {}

NbParams(CubeBrowser, execute='next')

In [ ]:
# Finally we can declare a cell which uses the settings defined via the widgets to render the requested plot.
# We simply look up the correct cube, convert it to the desired Element type and then display it with the requested options.

key = (CubeBrowser.quantity, CubeBrowser.element)
if key in CubeBrowser.cache:
    converted = CubeBrowser.cache[key]
else:
    holocube = cube_loader.cubes[cube_browser.quantity]
    converted = holocube.to(cube_browser.element, ['grid_longitude', 'grid_latitude'], dynamic=True)
    CubeBrowser.cache[key] = converted
styled = converted(style={cube_browser.element.name: dict(cmap=cube_browser.cmap)})
projection = projections[cube_browser.projection]()
(styled * hc.GeoFeature(cf.COASTLINE)(plot=dict(scale='50m'))(plot=dict(projection=projection)))